## Query the Planet with EOLearn and Sentinel Data

The availability of open Earth observation (EO) data through the Copernicus and Landsat programs represents an unprecedented resource for many EO applications, ranging from ocean and land use and land cover monitoring, disaster control, emergency services and humanitarian relief. Given the large amount of high spatial resolution data at high revisit frequency, techniques able to automatically extract complex patterns in such spatio-temporal data are needed.

[`eo-learn`](https://eo-learn.readthedocs.io/en/latest/) is a collection of open source Python packages that have been developed to seamlessly access and process spatio-temporal image sequences acquired by any satellite fleet in a timely and automatic manner. eo-learn is easy to use, it’s design modular, and encourages collaboration – sharing and reusing of specific tasks in a typical EO-value-extraction workflows, such as cloud masking, image co-registration, feature extraction, classification, etc. Everyone is free to use any of the available tasks and is encouraged to improve them, develop new ones and share them with the rest of the community.

We'll walk through the basics of `eo-learn` and an example EO analysis workflow.

![eo-learn workflow](eo-learn-illustration.png)

## Boring Setup

In [ ]:
# Jupyter notebook related commands
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# set our Sentinel Hub Instance ID (Workshop only)
# Sign-up at afterwards at https://www.sentinel-hub.com/ to continue using
!sentinelhub.config --instance_id eb3fe65f-23bf-4481-8d58-f7b8a8939b39

## EO Learn Fundamentals

`eo-learn` has three fundamental classes `EOPatch`, `EOTask` and `EOWorkflow` which are helpful to understand before undertaking a larger project

### EOPatch
- an **abstract representation of remotely sensed data over a single area**; the base data class of `eo-learn`
- a collection of 10 python dicts representing various types of data along with a bounding box and timestamps
- provides useful dimension and type checking, a consistent interface for later processing, and some convenience functions

Find more details in the [documentation](https://eo-learn.readthedocs.io/en/latest/examples/core/CoreOverview.html#EOPatch)

In [ ]:
# first create an empty patch, it is instantiated with all the data types
# each FeatureType (data, mask, scalar, ...) requires a specific type of data
from eolearn.core import EOPatch

patch = EOPatch()
patch

In [ ]:
# the first Featuretype 'data' should be a numpy array with four dimensions, time x height x width x channel depth, to represent a time stack of remotely sensed data 

# create an example of satellite data with 5 dates, 10x10 pixels, and 3 bands
import numpy as np
example_bands = np.zeros((5, 10, 10, 3), dtype=np.uint16)
patch.data['example'] = example_bands

In [ ]:
# EOPatches will throw an error if you give data of the wrong shape to a certain FeatureType

# here's satellite data with no time component
timeless_data = np.zeros((10, 10, 13), dtype=np.uint16)
try:
    patch.data['timeless_example'] = timeless_data
except ValueError as e:
    print(e)

In [ ]:
# there is a different FeatureType available for data without a time component
patch.data_timeless['timeless_example'] = timeless_data

In [ ]:
# try to add data to other FeatureTypes, use https://eo-learn.readthedocs.io/en/latest/examples/core/CoreOverview.html#EOPatch for reference
import geopandas as gpd
from datetime import datetime
from sentinelhub import BBox, CRS

patch.mask['shadow masks'] = np.zeros((5, 6, 6, 2), dtype=np.bool)
patch.scalar['weather data'] = np.zeros((5, 3), dtype=np.float)
patch.label['classification'] = np.zeros((5, 1), dtype=np.bool)


vector = gpd.read_file('eastern_france.geojson')
vector.insert(1, 'TIMESTAMP', datetime.now())
patch.vector['land use vectors'] = vector

patch.mask_timeless['district_boundaries_raster'] = np.zeros((6, 6, 2), dtype=np.uint8)
patch.scalar_timeless['single patch value'] = np.array([5.])
patch.label_timeless['another patch value'] = np.array([True])
patch.vector_timeless['region_boundaries'] = gpd.read_file('eastern_france.geojson')

patch.meta_info = dict(any_property={ 'a': 5, 'b': 3 })
patch.bbox = BBox([24.5, 45.5, 25.5, 46.5], crs=CRS.WGS84)
patch.timestamp = [datetime.now()]

patch.get_features()

In [ ]:
# we can save and load our patch; check out the file structure it creates
patch.save('example_patch')
patch2 = patch.load('example_patch')
patch2.get_features()

### EOTask
- an **abstract representation of an operation over remotely sensed data** over a single area; the base processing class of `eo-learn`
- must have an `execute` method which generally accepts an `EOPatch` instance as its first argument
- provides consistency of operation, chainable, predictable outputs
- many `EOTasks` are already available in the library (60+ at the time of writing)

In [ ]:
# let's start with a simple one to understand the concept
# AddFeature performs the same operation we were doing before but in EOTask form
from eolearn.core import AddFeature, FeatureType

# specify the feature type and name, then instantiate the task (but not call it yet)
feature = (FeatureType.DATA, 'bands')
add_feature = AddFeature(feature)

# now we can execute the task, on our previous patch
data = np.zeros((5, 100, 100, 13))
patch = add_feature.execute(patch, data)
patch

In [ ]:
# now let's try a more substantial task
from eolearn.io import S2L1CWCSInput

# we'll make a new empty EOPatch over the conference for this
patch = EOPatch()
patch.bbox = BBox([26.08, 44.42, 26.14, 44.46], crs=CRS.WGS84)

# now we can load Sentinel-2 data into the patch
# first instantiate the task with a Sentinel Hub layer and our desired resolution
load_s2 = S2L1CWCSInput('TRUE-COLOR-S2-L1C', resx='10m', resy='10m')

# then call the task's execute method with our patch and a time interval
load_s2.execute(eopatch=patch, time_interval=('2019-08-23', '2019-08-27'))


In [ ]:
# since the data is a numpy array, let's try visualizing it with matplotlib
from matplotlib import pyplot as plt
plt.imshow(patch.data['TRUE-COLOR-S2-L1C'][0])

In [ ]:
# try with other resolutions, layers, date ranges or areas
# also notice the additional information that the task adds to mask, meta_info and timestamp
patch = EOPatch()
patch.bbox = BBox((-77.04, 38.90, -77, 38.96), crs=CRS.WGS84)

load_s2 = S2L1CWCSInput('BANDS-S2-L1C', resx='20m', resy='20m') # all bands
load_s2.execute(eopatch=patch, time_interval=('2019-08-01', '2019-08-06'))

In [ ]:
# now let's add some vector data, load a local file containing buildings
buildings = gpd.read_file('bucharest_buildings.geojson')

In [ ]:
# we can add this file as either a vector (directly) or try transforming it to a raster
from eolearn.geometry import VectorToRaster
vtr_task = VectorToRaster(
    buildings,
    values=1,
    raster_shape=(FeatureType.DATA, 'TRUE-COLOR-S2-L1C'), # match the shape of our previous data
    raster_feature=(FeatureType.MASK_TIMELESS, 'BUILDING-DATA') # name our new layer
)

vtr_task.execute(eopatch=patch)

In [ ]:
# now look at the buildings (also congrats, you're 50% of the way to being a geospatial machine learning engineer)
plt.imshow(patch.mask_timeless['BUILDING-DATA'].squeeze(2))

In [ ]:
# we can also implement our own tasks, they need to:
# - inherit from the EOTask class
# - implement an __init__ method
# - implement an execute method which accepts an EOPatch as the first argument

# here's a template, try it out!
from eolearn.core import EOTask
    
# create a task for calculating a median pixel value
class MedianPixel(EOTask):
    """
    The task returns a pixelwise median value from a time-series and stores the results in a 
    timeless data array.
    """
    def __init__(self, input_feature, output_feature):
        self.input_feature = input_feature
        self.output_feature = output_feature

    def execute(self, eopatch):
        median = np.median(eopatch[self.input_feature[0]][self.input_feature[1]], axis=0)
        eopatch.add_feature(*self.output_feature, median)
        return eopatch
    
median_task = MedianPixel((FeatureType.DATA, 'time_series'), (FeatureType.DATA_TIMELESS, 'median'))
median_task.execute(patch)

In [ ]:
# finally let's save our patch
from eolearn.core import SaveToDisk, OverwritePermission

save = SaveToDisk('example', overwrite_permission=OverwritePermission.OVERWRITE_PATCH)

save.execute(eopatch=patch, eopatch_folder='this_patch')

## EOWorkflow

- a **combination of `EOTask` objects** meant to accomplish a larger project
- builds a computational graph object to ensure all patches are processed in the same fashion

In [ ]:
# rather than call .execute directly, let chain our tasks together
from eolearn.core import LinearWorkflow

workflow = LinearWorkflow(load_s2, vtr_task, save)

In [ ]:
# now we can call this on any number of patches and pass the execution arguments to the workflow
patch = EOPatch()
bbox = BBox([26.08, 44.42, 26.14, 44.46], crs=CRS.WGS84)
patch.bbox = bbox
patches = [patch]

for patch in patches:
    workflow.execute({
        load_s2: { 'time_interval': ('2019-08-23', '2019-08-27'), 'bbox': bbox },
        vtr_task: {},
        save: { 'eopatch_folder': 'patch_idx' }
    }).eopatch()
    


## Practice

Take some time to experiment with different tasks and parameters before starting a larger workflow